# Interactuando con la base de datos NoSQL Amazon DynamoDB

En este laboratorio, trabajarás con DynamoDB como una base de datos clave-valor y aplicarás algunas operaciones de Crear, Leer, Actualizar y Eliminar (CRUD) en esta base de datos NoSQL.

*Nota*:
- El laboratorio contiene enlaces a recursos externos. Siempre puedes echar un vistazo a estos recursos durante la sesión de laboratorio, pero no se espera que abras y leas cada enlace durante la sesión de laboratorio. Si deseas profundizar tu comprensión, puedes revisar los recursos vinculados después de que hayas terminado con el laboratorio.
- El laboratorio contiene 4 partes opcionales que puedes optar por omitir.

Para abrir el cuaderno de soluciones, sigue estos pasos:
- Ve al menú principal y selecciona `Archivo -> Preferencias -> Configuración`.
- Haz clic en `Editor de texto` a la izquierda, luego desplázate hacia abajo hasta la sección `Excluir archivos`.
- Elimina la línea `**/C2_W1_Lab_2_DynamoDB_Solution.ipynb`. El archivo ahora aparecerá en el explorador.
- Puedes cerrar la pestaña `Configuración`.


# Tabla de Contenidos
- [ 1 - Importar Paquetes](#1)
- [ 2 - Explorar los Datos](#2)
- [ 3 - Crear las Tablas de DynamoDB](#3)
  - [ Ejercicio 1](#ex01)
- [ 4 - Cargar Datos en las Tablas](#4)
  - [ 4.1 - Cargar Datos Elemento por Elemento](#4-1)
    - [ Ejercicio 2](#ex02)
  - [ 4.2 - Cargar Datos como un Lote de Elementos](#4-2)
    - [ Ejercicio 3](#ex03)
- [ 5 - Leer Datos de las Tablas](#5)
  - [ 5.1 - Escanear la Tabla Completa](#5-1)
    - [ Ejercicio 4](#ex04)
  - [ 5.2 - Leer un Único Elemento](#5-2)
    - [ Ejercicio 5](#ex05)
  - [ 5.3 - Consultar Elementos que Comparten la Misma Clave de Partición](#5-3)
    - [ Ejercicio 6](#ex06)
    - [ Ejercicio 7](#ex07)
  - [ 5.4 - Filtrar los Escaneos de la Tabla](#5-4)
- [ 6 - Insertar y Actualizar Datos](#6)
  - [ 6.1 - Insertar Datos](#6-1)
  - [ 6.2 - Actualizar Datos](#6-2)
    - [ Ejercicio 8](#ex08)
    - [ Ejercicio 9](#ex09)
- [ 7 - Eliminar Datos](#7)
  - [ Ejercicio 10](#ex10)
- [ 8 - Transacciones - Opcional](#8)
  - [ Ejercicio 11](#ex11)
- [ 9 - Limpieza](#9)


<a id='1'></a>
## 1 - Importar paquetes

Primero, importemos algunos paquetes. Entre estos paquetes, puedes encontrar `boto3`, que es el kit de desarrollo de software (SDK) de AWS para Python que te permite interactuar con varios servicios de AWS utilizando código Python. Con `boto3`, puedes acceder programáticamente a recursos de AWS como instancias EC2, buckets S3, tablas de Amazon DynamoDB y más. Te proporciona una interfaz simple e intuitiva para gestionar e integrar servicios de AWS en tus aplicaciones de Python de manera eficiente.

Para obtener más información sobre cada uno de los métodos que utilizarás a lo largo de este laboratorio, puedes consultar la [documentación de boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html).


In [ ]:
import decimal
import json
import logging
from typing import Any, Dict, List

import boto3
from botocore.exceptions import ClientError
from IPython.display import HTML

Definamos la siguiente variable que utilizarás a lo largo de este laboratorio.


In [ ]:
COURSE_PREFIX = 'de-c2w1-dynamodb'

<a id='2'></a>
## 2 - Explorar los Datos

El conjunto de datos que utilizarás en este laboratorio es el conjunto de datos de muestra del [Guía del desarrollador de Amazon DynamoDB](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/AppendixSampleTables.html#AppendixSampleData) ([archivo zip del conjunto de datos](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/samples/sampledata.zip)). Los datos de muestra consisten en 4 archivos JSON que puedes encontrar en la carpeta `data/aws_sample_data`:
- `ProductCatalog`: Catálogo de productos que contiene información sobre algunos productos como el ID del producto y sus características.
- `Forum`: Información sobre algunos foros de AWS donde los usuarios publican preguntas o inician un hilo (es decir, una conversación) sobre los servicios de AWS. La información incluye el nombre del foro y el número total de hilos, mensajes y vistas en cada foro.
- `Thread`: Información sobre cada hilo del foro (es decir, una conversación), como el asunto del hilo, el mensaje del hilo, el número total de vistas y respuestas al hilo dado, y quién publicó por último en el hilo.
- `Reply`: Información sobre las respuestas de cada hilo, como la hora de la respuesta, el mensaje de la respuesta y el usuario que publicó la respuesta.

En este laboratorio, crearás 4 tablas de DynamoDB (`de-c2w1-dynamodb-ProductCatalog`, `de-c2w1-dynamodb-Forum`, `de-c2w1-dynamodb-Thread`, `de-c2w1-dynamodb-Reply`) y cargarás en cada una los datos del archivo JSON correspondiente.

*Nota*: Si revisas el contenido de cada archivo JSON, notarás el uso de letras como N, S, B. Estas son conocidas como [*Descriptores de tipo de datos*](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/HowItWorks.NamingRulesDataTypes.html#HowItWorks.DataTypeDescriptors) que indican a DynamoDB cómo interpretar el tipo de cada campo. Hablaremos más sobre esto más adelante en este laboratorio.


<a id='3'></a>
## 3 - Crear las tablas de DynamoDB

**¿Qué es una tabla de DynamoDB?**

La base de datos de DynamoDB es una tienda de clave-valor que almacena un conjunto de pares clave-valor. Digamos que tienes un conjunto de elementos clave-valor donde cada elemento representa un producto. Cada elemento se caracteriza por una clave única (ID de producto) y tiene un conjunto de atributos correspondientes (el valor de la clave). DynamoDB almacena estos datos clave-valor en una tabla donde cada fila contiene los atributos de un producto y utiliza la clave para identificar de forma única cada fila. Esta tabla es diferente de las tablas relacionales porque no tiene un esquema, lo que significa que ni los atributos ni sus tipos de datos necesitan definirse de antemano. Cada elemento puede tener sus propios atributos distintos. Por ejemplo, en la tabla de productos que crearás en esta sección, tendrás un elemento que representa un libro (Título, Autores, ISBN, Precio) y otro elemento que representa una bicicleta (Tipo de bicicleta, Marca, Precio, Color), ambos almacenados en la misma tabla de DynamoDB.


**¿Cuál es la clave primaria de una tabla de DynamoDB?**

Cuando creas una tabla de DynamoDB, necesitas especificar la clave primaria que es la clave que identifica de forma única cada ítem. La clave primaria puede ser una clave simple - clave de partición - o una clave primaria compuesta - clave de partición y clave de ordenación.
- clave de partición (clave simple): Por ejemplo, en las tablas de productos, se utilizará el ID del producto como clave primaria ya que identifica de forma única cada producto. Para DynamoDB, esta clave primaria simple se llama clave de partición porque DynamoDB la utiliza como entrada a una función hash. La salida de la función hash determina la partición (almacenamiento físico interno) en la que se almacenará el ítem.
- clave de partición y clave de ordenación (clave compuesta): En esta clave compuesta, dos ítems pueden tener la misma clave de partición pero deben tener diferentes claves de ordenación para que la clave compuesta aún pueda identificar de forma única cada ítem. DynamoDB utilizará la clave de partición para determinar en qué partición se almacenará el ítem. Todos los ítems con el mismo valor de clave de partición se almacenan juntos, en orden ordenado por el valor de la clave de ordenación.

Puedes aprender más sobre los componentes principales de DynamoDB [aquí](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/HowItWorks.CoreComponents.html).


**¿Cómo crearás las tablas?**

Utilizarás el método [DynamoDB create_table()](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb/client/create_table.html). Este método espera 3 parámetros requeridos:
* `TableName`: el nombre de la tabla.
* `KeySchema`: un array de los atributos que conforman la clave primaria de una tabla. Para cada elemento en este array, necesitas especificar: `AttributeName`: el nombre del atributo, y `KeyType`: el rol que asumirá el atributo clave (`HASH` si es una clave de partición y `RANGE` si es una clave de ordenamiento). Por ejemplo,
  ```
  'KeySchema'= [
      {'AttributeName': 'ForumName', 'KeyType': 'HASH'}, 
      {'AttributeName': 'Subject', 'KeyType': 'RANGE'}
  ]
  ```
* `AttributeDefinitions`: un array que describe los atributos que conforman la clave primaria. Para cada elemento en este array, necesitas especificar `AttributeName` y `AttributeType`: el tipo de datos del atributo (S: String, N: Number, B: Binary,...). Por ejemplo, 
  ```
  'AttributeDefinitions': [
      {'AttributeName': 'ForumName', 'AttributeType': 'S'},
      {'AttributeName': 'Subject', 'AttributeType': 'S'}
  ]
  ```
Hay un parámetro adicional que puedes especificar si no deseas pagar por DynamoDB basado en la demanda y prefieres elegir el modo provisionado:
* `ProvisionedThroughput`: un diccionario que especifica la capacidad de lectura/escritura (o rendimiento) para una tabla especificada. Consta de dos elementos:
  - `ReadCapacityUnits`: el número máximo de lecturas consistentes fuertes consumidas por segundo;
  - `WriteCapacityUnits`: el número máximo de escrituras consumidas por segundo.

En este laboratorio, crearás 4 tablas, y para cada tabla, necesitas especificar los parámetros que acabamos de enumerar aquí. Para facilitar tu acceso a las propiedades de cada tabla a lo largo de este cuaderno, creamos los siguientes diccionarios que especifican las propiedades para cada tabla.


In [ ]:
capacity_units = {'ReadCapacityUnits': 10, 'WriteCapacityUnits': 5}

product_catalog_table = {'table_name': f'{COURSE_PREFIX}-ProductCatalog',
                         'kwargs': {
                             'KeySchema': [{'AttributeName': 'Id', 'KeyType': 'HASH'}],
                             'AttributeDefinitions': [{'AttributeName': 'Id', 'AttributeType': 'N'}],
                             'ProvisionedThroughput': capacity_units
                         }
                        }

forum_table = {'table_name': f'{COURSE_PREFIX}-Forum',
                'kwargs': {
                    'KeySchema': [{'AttributeName': 'Name', 'KeyType': 'HASH'}],
                    'AttributeDefinitions': [{'AttributeName': 'Name', 'AttributeType': 'S'}],
                    'ProvisionedThroughput': capacity_units
                }
              }

thread_table = {'table_name': f'{COURSE_PREFIX}-Thread',
                'kwargs': {
                    'KeySchema': [{'AttributeName': 'ForumName', 'KeyType': 'HASH'}, 
                                  {'AttributeName': 'Subject', 'KeyType': 'RANGE'}],
                    'AttributeDefinitions': [{'AttributeName': 'ForumName', 'AttributeType': 'S'},
                                             {'AttributeName': 'Subject', 'AttributeType': 'S'}],
                    'ProvisionedThroughput': capacity_units
                }
               }

reply_table = {'table_name': f'{COURSE_PREFIX}-Reply',
                'kwargs': {
                    'KeySchema': [{'AttributeName': 'Id', 'KeyType': 'HASH'}, 
                                  {'AttributeName': 'ReplyDateTime', 'KeyType': 'RANGE'}],
                    'AttributeDefinitions': [{'AttributeName': 'Id', 'AttributeType': 'S'},
                                             {'AttributeName': 'ReplyDateTime', 'AttributeType': 'S'}],
                    'ProvisionedThroughput': capacity_units
                }
              }

Las tablas de hilo y respuesta utilizarán ambas una clave primaria compuesta, y las tablas de producto y foro utilizarán una clave primaria simple.


*Nota:* Para interactuar con AmazonDynamoDB a lo largo de este cuaderno, vas a crear un objeto cliente `boto3`. Este objeto te permite hacer solicitudes de API directamente a los servicios de AWS para crear, eliminar o modificar recursos. Cuando creas un objeto cliente `boto3`, deberás especificar los servicios de AWS con los que deseas interactuar, y luego, con el objeto cliente creado, puedes llamar a métodos para realizar varias operaciones en ese recurso.


<a id='ex01'></a>
### Ejercicio 1

Para crear las 4 tablas, utilizarás la función `create_table_db()` proporcionada en la siguiente celda. Esta función llama al método `DynamoDB create_table()`, y toma dos argumentos:
* `table_name`: el nombre de la tabla;
* `kwargs`: Un diccionario que especifica los argumentos adicionales para `DynamoDB create_table()` como `KeySchema`, `AttributeDefinitions` y `ProvisionedThroughput` como se muestra en la celda anterior. `**kwargs` significa que los elementos en el diccionario se desempaquetan en una secuencia de argumentos.

En este primer ejercicio, deberás reemplazar `None` con los valores apropiados.


In [ ]:
def create_table_db(table_name: str, **kwargs):
    client = boto3.client("dynamodb")
    ### START CODE HERE ### (~ 1 line of code)
    response = client.create_table(TableName=table_name, **kwargs) # @REPLACE EQUALS client.create_table(TableName=None, None)
    ### END CODE HERE ###

    waiter = client.get_waiter("table_exists")
    waiter.wait(TableName=table_name)

    return response

Ahora que la función `create_table_db()` está lista, puedes probarla creando la tabla `ProductCatalog`. La ejecución debería tomar menos de un minuto.


In [ ]:
response = create_table_db(table_name=product_catalog_table['table_name'], **product_catalog_table["kwargs"]) 
print(response)

##### __Salida Esperada__

**Nota**: Los componentes de fecha y hora pueden ser diferentes.

```
{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'Id', 'AttributeType': 'N'}], 'TableName': 'de-c2w1-dynamodb-ProductCatalog', 'KeySchema': [{'AttributeName': 'Id', 'KeyType': 'HASH'}], 'TableStatus': 'CREATING', 'CreationDateTime': datetime.datetime(2024, 2, 14, 6, 42, 38, 872000, tzinfo=tzlocal()), 'ProvisionedThroughput': {'NumberOfDecreasesToday': 0, 'ReadCapacityUnits': 10, 'WriteCapacityUnits': 5}, 'TableSizeBytes': 0, 'ItemCount': 0, 'TableArn': 'arn:aws:dynamodb:us-east-1:631295702609:table/de-c2w1-dynamodb-ProductCatalog', 'TableId': '639df373-f498-4a2d-9851-6c6f6c26d908', 'DeletionProtectionEnabled': False}, 'ResponseMetadata': {'RequestId': 'OJ9GC0U10JH5ILK020C4PM094VVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 14 Feb 2024 06:42:38 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '557', 'connection': 'keep-alive', 'x-amzn-requestid': 'OJ9GC0U10JH5ILK020C4PM094VVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '1500356689'}, 'RetryAttempts': 0}}
```


Ejecuta el siguiente comando para crear las otras tres tablas. La creación de todas las tablas puede tardar alrededor de 2 minutos.


In [ ]:
for dynamodb_tab in [forum_table, thread_table, reply_table]:
    response = create_table_db(dynamodb_tab["table_name"], **dynamodb_tab["kwargs"])
    print(response)

Ejecuta el siguiente código para obtener el enlace a la consola de AWS.

*Nota*: Por razones de seguridad, la URL para acceder a la consola de AWS caducará cada 15 minutos, pero cualquier recurso de AWS que hayas creado seguirá estando disponible durante el período de 2 horas. Si necesitas acceder a la consola después de 15 minutos, vuelve a ejecutar esta celda de código para obtener un nuevo enlace activo.


In [ ]:
with open('../.aws/aws_console_url', 'r') as file:
    aws_url = file.read().strip()

HTML(f'<a href="{aws_url}" target="_blank">GO TO AWS CONSOLE</a>')

Ve al panel de control de AWS, busca **DynamoDB**, haz clic en Tablas a la izquierda y verifica que las tablas hayan sido creadas.

*Nota:* Si ves la ventana como en la siguiente captura de pantalla, haz clic en el enlace **logout**, cierra la ventana y vuelve a hacer clic en el enlace de la consola.

![AWSLogout](images/AWSLogout.png)

<a id='4'></a>
## 4 - Cargar datos en las tablas

Ahora cargarás datos en cada tabla a partir de los siguientes archivos JSON:
* `Forum.json`
* `ProductCatalog.json`
* `Reply.json`
* `Thread.json`

Puedes cargar los datos elemento por elemento o como un lote de elementos. Vamos a explorar cada opción.


<a id='4-1'></a>
### 4.1 - Cargar datos elemento por elemento

Para cargar datos elemento por elemento, utilizarás el método: [DynamoDB put_item()](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb/client/put_item.html). Este método espera dos argumentos requeridos (1) el nombre de la tabla y (2) el elemento que necesitas agregar. El elemento debe ser un diccionario que contenga los atributos del elemento (y, lo más importante, el valor de su clave primaria), por ejemplo, aquí está el formato de cómo debería lucir el elemento (un elemento en la tabla de respuestas):
```
item = {
        "Id": {
            "S": "Amazon DynamoDB#DynamoDB Thread 1"
            },
        "ReplyDateTime": {
             "S": "2015-09-15T19:58:22.947Z"
             },
        "Message": {
            "S": "Texto de la respuesta 1 del hilo 1 de DynamoDB"
        },
        "PostedBy": {
            "S": "Usuario A"
        }
}
```
Esta estructura JSON se ve de la siguiente manera:

```JSON
{
    "<NombreAtributo>": {
        "<TipoDato>": "<Valor>"
    },
    "<AtributoLista>": {
        "<TipoDato>": [
            {
                "<TipoDato>": "<Valor1>"
            },
            {
                "<TipoDato>": "<Valor2>"
            }]
    }    
}
```
se llama JSON de Marshal. Esto es similar a un archivo JSON regular, pero también incluye los tipos de cada valor. Los marcadores de posición `<TipoDato>` especifican el tipo de datos del valor correspondiente; puedes obtener más información sobre las convenciones de tipos de datos para DynamoDB en la [documentación](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/HowItWorks.NamingRulesDataTypes.html#HowItWorks.DataTypeDescriptors). La buena noticia es que todos los elementos proporcionados en los archivos JSON de muestra ya están en este formato esperado para `DynamoDB put_item()`.


En esta sección, se te proporcionan dos funciones:
- `read_data()`: lee un archivo JSON de muestra y devuelve los elementos como un diccionario de Python;
- `put_item_db()`: Esta función toma como argumentos el nombre de la tabla y los detalles del elemento como un diccionario de Python, llama a `DynamoDB put_item()` y le pasa el nombre de la tabla y el elemento.

En el ejercicio de esta sección, solo necesitas reemplazar `None` dentro de la función `put_item_db()`. No necesitas modificar nada dentro de `read_data()`. Utilizarás la función `read_data()` para leer todos los elementos del archivo JSON, y luego utilizarás la función `put_item_db()` para cargar cada elemento en una tabla de DynamoDB dada.


In [ ]:
def read_data(file_path: str) -> Dict[str, Any]:
    with open(file_path, "r") as json_file:
        items = json.load(json_file)
    return items

<a id='ex02'></a>
### Ejercicio 2

En este ejercicio, necesitas reemplazar `None` con los valores apropiados:
1. Crea un objeto Cliente (ver el código en el ejercicio anterior).
2. Utiliza el método `client.put_item()` del objeto `client` para cargar los datos, el cual espera tres argumentos: `TableName`, el `Item` a cargar y algunos argumentos de palabras clave.


In [ ]:
def put_item_db( table_name: str, item: Dict[str, Any], **kwargs):
    ### START CODE HERE ### (~ 2 lines of code)
    client = boto3.client("dynamodb") # @REPLACE EQUALS None
    response = client.put_item(TableName=table_name, Item=item, **kwargs) # @REPLACE EQUALS client.put_item(TableName=None, Item=None, None)
    ### END CODE HERE ###

    return response

Ahora, carguemos los elementos de los archivos `ProductCatalog` y `Thread` uno por uno en las tablas correspondientes.


In [ ]:
for dynamodb_tab in [product_catalog_table, thread_table]:
    file_name = dynamodb_tab['table_name'].split('-')[-1]    
    items = read_data(file_path=f'./data/aws_sample_data/{file_name}.json')
    
    for item in items[dynamodb_tab["table_name"]]:
        put_item_db(table_name=dynamodb_tab["table_name"], item=item['PutRequest']['Item'])

<a id='4-2'></a>
### 4.2 - Cargar datos como un lote de elementos

Ahora, crearás la función `batch_write_item_db()` que llama al método [DynamoDB batch_write_item()](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb/client/batch_write_item.html). Este último método te permite poner o eliminar múltiples elementos en una o más tablas.

Nuevamente, tendrás que leer los dos archivos JSON `Reply` y `Forum` y luego cargar los elementos en las tablas. Carguemos los datos en las tablas `Reply` y `Forum`.


<a id='ex03'></a>
### Ejercicio 3

En este ejercicio, necesitas reemplazar `None` con los valores apropiados:
1. Crea el objeto Cliente;
2. Llama al método `client.batch_write_item()` del objeto `client`. Debería recibir los elementos que necesitan ser cargados y algunos argumentos de palabra clave. Asume que la entrada `items` está en el formato correcto que `batch_write_item()` espera (el formato de los elementos almacenados en los archivos JSON de muestra es exactamente el formato que `batch_write_item()` espera. Para más información, puedes consultar la documentación [aquí](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb/client/batch_write_item.html)).


In [ ]:
def batch_write_item_db(items: Dict[str, Any], **kwargs):
    ### START CODE HERE ### (~ 2 lines of code)
    client = boto3.client("dynamodb") # @REPLACE EQUALS None
    response = client.batch_write_item(RequestItems=items, **kwargs) # @REPLACE EQUALS client.batch_write_item(RequestItems=None, None)
    ### END CODE HERE ###
    
    return response

Ahora, vamos a leer los datos de los archivos de muestra JSON: `Reply` y `Forum` y luego cargar los elementos como un lote en las tablas correspondientes.


In [ ]:
for dynamodb_tab in [reply_table, forum_table]:
    file_name = dynamodb_tab['table_name'].split('-')[-1]    
    items = read_data(file_path=f'./data/aws_sample_data/{file_name}.json')
    response = batch_write_item_db(items=items)
    print(response)

##### __Salida Esperada__

**Nota**: Los componentes de fecha y hora pueden ser diferentes.

```
{'UnprocessedItems': {}, 'ResponseMetadata': {'RequestId': '4P678E81BOHRCUN82FFREOTC8NVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 14 Feb 2024 06:44:36 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '23', 'connection': 'keep-alive', 'x-amzn-requestid': '4P678E81BOHRCUN82FFREOTC8NVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '4185382651'}, 'RetryAttempts': 0}}
{'UnprocessedItems': {}, 'ResponseMetadata': {'RequestId': 'R53NDPHFEH0UEL0MG6PFG8FEJRVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 14 Feb 2024 06:44:36 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '23', 'connection': 'keep-alive', 'x-amzn-requestid': 'R53NDPHFEH0UEL0MG6PFG8FEJRVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '4185382651'}, 'RetryAttempts': 0}
}
```


<a id='5'></a>
## 5 - Leer datos de las tablas

En esta sección, experimentarás con varios enfoques para leer datos de las tablas de DynamoDB.


<a id='5-1'></a>
### 5.1 - Escanear la tabla completa

Puede realizar una operación `DynamoDB scan()` en una tabla de DynamoDB que escanea completamente la tabla y devuelve los elementos en fragmentos de 1MB. Escanear es la forma más lenta y costosa de obtener datos de DynamoDB. Primero exploremos este enfoque.


<a id='ex04'></a>
### Ejercicio 4

En este ejercicio, necesitas reemplazar `None` con los valores apropiados:
1. Crea el objeto Cliente `client`.
2. Llama al método `client.scan()` del objeto `client`. Debería recibir el nombre de la tabla y argumentos de palabra clave. En la [documentación de DynamoDB boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html), busca el método `scan` para verificar qué parámetros toma.


In [ ]:
def scan_db(table_name: str, **kwargs):
    ### START CODE HERE ### (~ 2 lines of code)
    client = boto3.client("dynamodb") # @REPLACE EQUALS None
    response = client.scan(TableName=table_name, **kwargs) # @REPLACE EQUALS client.scan(TableName=None, None)
    ### END CODE HERE ###
    
    return response

Realicemos un escaneo completo en la tabla `ProductCatalog`:


In [ ]:
response = scan_db(product_catalog_table['table_name'])
print(f"Queried data for table {product_catalog_table['table_name']}:\n{response}")

Puedes ver que los datos devueltos tienen la misma estructura de entrada que el método `DynamoDB put_item()` espera, que es el Marshal JSON. Marshal JSON es diferente del formato JSON habitual que se ve así:

```JSON
{
    "AttributeName": "Value",
    "ListAttribute": [
        "Value1",
        "Value2"
    ]
}
```

El formato JSON habitual es el formato típico que encontrarás en la vida real, ya que se puede analizar fácilmente en Diccionarios de Python. Por lo tanto, es posible que necesites convertir la salida devuelta por el método `DynamoDB scan()` al formato JSON habitual, o puede que necesites convertir datos que estén en el formato JSON habitual a Marshal JSON antes de insertarlos en una tabla de DynamoDB. La siguiente parte opcional te muestra cómo puedes convertir datos en Marshal JSON al formato JSON habitual. Puedes probar la parte opcional o siéntete libre de omitirla.


#### Parte opcional - 1 (Deserializando Marshal JSON)


Ahora, si deseas procesar los datos devueltos por las operaciones de DynamoDB con Python, debes convertir el formato de los datos al JSON habitual. `boto3` proporciona algunas utilidades para ayudarte con este proceso.
Para convertir los datos de `ProductCatalog` devueltos por el método de escaneo a un formato JSON regular que se pueda utilizar en diccionarios de Python, puedes utilizar la función `data_deserializer()` proporcionada a continuación que toma como entrada los datos en formato Marshal JSON. Esta función consta de lo siguiente:

1. Una instanciación de recurso `boto3`: [Resources](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/resources.html) es una clase de abstracción de nivel superior construida sobre Cliente que se utiliza para representar recursos de AWS como objetos de Python, proporcionando de esta manera una interfaz Pythonica y orientada a objetos. Con ese recurso, puedes crear un objeto deserializador llamando al método `TypeDeserializer()`.
2. Luego puedes utilizar el objeto `deserializer` para llamar al método `deserializer.deserialize()` y aplicarlo a cada valor para convertirlo en su versión deserializada. (*Nota*: si el valor devuelto de `deserializer.deserialize(v)` es una instancia de `decimal.Decimal`, debes convertirlo a float. Este proceso de comprobar si el valor devuelto es una instancia de `decimal.Decimal` debe hacerse porque, de forma predeterminada, los valores numéricos en DynamoDB se deserializan como decimales, los cuales deben manejarse adecuadamente si deseas trabajar con el resultado; la forma más sencilla es convertirlos directamente al tipo de dato float).

La función a continuación utiliza comprensión de diccionarios para iterar a través de los elementos del diccionario.


In [ ]:
def data_deserializer(data: Dict[str, Any]):
    boto3.resource("dynamodb")

    deserializer = boto3.dynamodb.types.TypeDeserializer()

    deserialized_data = {
        k: (
            float(deserializer.deserialize(v))
            if isinstance(deserializer.deserialize(v), decimal.Decimal)
            else deserializer.deserialize(v)
        )
        for k, v in data.items()
    }

    return deserialized_data

Ejecuta el método sobre la respuesta anterior para ver la diferencia en el formato.


In [ ]:
for item in response['Items']:
    print(f"DynamoDB returned Marshal JSON:\n{item}")
    print(f"Deserialized python dictionary:\n {data_deserializer(item)}")

Si quieres entender más sobre el proceso de transformación entre Marshall JSON y diccionarios JSON/Python, puedes encontrar herramientas como esta que te permitirán practicar con ellos. También puedes echar un vistazo a la documentación de `boto3` para ver cómo se implementan TypeSerializer y TypeDeserializer.


#### Fin de la Parte Opcional - 1


<a id='5-2'></a>
### 5.2 - Leer un solo elemento

El método `DynamoDB scan()` devuelve todos los elementos en una tabla. Si desea leer un solo elemento, podría usar el método `DynamoDB get_item()`. Este método espera el nombre de la tabla y la clave primaria del elemento solicitado. Es la forma más barata y rápida de obtener datos de DynamoDB.


<a id='ex05'></a>
### Ejercicio 5

En la siguiente función, llama al método `client.get_item()` del objeto `client`. Debería recibir el nombre de la tabla, la clave y los argumentos de palabra clave. Para obtener más información sobre este método, puedes buscar el `get_item` en la [documentación](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html).


In [ ]:
def get_item_db(table_name, key: Dict[str, Any], **kwargs):
    client = boto3.client("dynamodb")

    try:
        ### START CODE HERE ### (~ 1 line of code)
        response = client.get_item(TableName=table_name, Key=key, **kwargs) # @REPLACE EQUALS client.get_item(TableName=None, Key=None, None)
        ### END CODE HERE ###
        
    except ClientError as e:
        error = e.response.get("Error", {})
        logging.error(
            f"Failed to query DynamoDB. Error: {error.get('Message')}"
        )
        response = {}
    
    return response

Obtener el elemento con Id 101 de la tabla `ProductCatalog`.


In [ ]:
response = get_item_db(table_name=product_catalog_table['table_name'], 
                    key={'Id': {'N': '101'}})
print(response)

##### __Salida Esperada__

**Nota**: Los componentes de fecha y hora pueden ser diferentes.

```
{'Item': {'Title': {'S': 'Título del Libro 101'}, 'EnPublicación': {'BOOL': True}, 'NúmeroDePáginas': {'N': '500'}, 'Dimensiones': {'S': '8.5 x 11.0 x 0.5'}, 'ISBN': {'S': '111-1111111111'}, 'Autores': {'L': [{'S': 'Autor1'}]}, 'Precio': {'N': '2'}, 'CategoríaDeProducto': {'S': 'Libro'}, 'Id': {'N': '101'}}, 'MetadataDeRespuesta': {'SolicitudId': '08VIS0M7LH396M766IOPU54E9JVV4KQNSO5AEMVJF66Q9ASUAAJG', 'CódigoDeEstadoHTTP': 200, 'EncabezadosHTTP': {'servidor': 'Servidor', 'fecha': 'Mié, 14 Feb 2024 06:44:53 GMT', 'tipo-de-contenido': 'application/x-amz-json-1.0', 'longitud-de-contenido': '263', 'conexión': 'mantener-viva', 'x-amzn-solicitudid': '08VIS0M7LH396M766IOPU54E9JVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '3181387427'}, 'IntentosDeReintento': 0}}
```


#### Parte opcional - 2 (Más opciones para los métodos de lectura)


Por defecto, una lectura de DynamoDB utilizará consistencia eventual. Una lectura consistente en DynamoDB es más barata que una lectura consistentemente fuerte. Varias opciones pueden ser añadidas a los métodos de lectura, algunas de las que se utilizan regularmente son:
- `ConsistentRead`: especifica que se requiere una lectura consistentemente fuerte de la tabla;
- `ProjectionExpression`: especifica qué atributos deben ser devueltos;
- `ReturnConsumedCapacity`: determina qué nivel de detalle sobre la capacidad consumida debe devolver la respuesta.

Puedes encontrar más información sobre los parámetros que acepta `DynamoDB.Client.get_item()` leyendo la [documentación](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb/client/get_item.html).

En el siguiente código, harás lo siguiente:
1. Establecerás el atributo `ConsistentRead` en `True` para asegurar lecturas consistentemente fuertes.
2. Especificarás que solo deseas recuperar los siguientes campos: `ProductCategory`, `Price` y `Title` utilizando el atributo `ProjectionExpression`.
3. Establecerás el atributo `ReturnConsumedCapacity` en `'TOTAL'`.
4. Consultarás el ítem con `Id=101` de la tabla `ProductCatalog`.


In [ ]:
kwargs = {'ConsistentRead': True,
          'ProjectionExpression': 'ProductCategory, Price, Title',
          'ReturnConsumedCapacity': 'TOTAL'}

response = get_item_db(table_name=product_catalog_table['table_name'], key={'Id': {'N': '101'}}, **kwargs)
print(response)

La solicitud anterior consumió 1.0 RCU porque este elemento es menor a 4KB. (RCU significa Unidad de Capacidad de Lectura: "Una unidad de capacidad de lectura representa una lectura consistente por segundo, o dos lecturas eventualmente consistentes por segundo, para un elemento de hasta 4 KB de tamaño", [referencia](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/provisioned-capacity-mode.html)).

Si ejecutas nuevamente el comando pero eliminas la opción ConsistentRead, puedes ver que las lecturas eventualmente consistentes consumen la mitad de capacidad.


In [ ]:
kwargs = {'ReturnConsumedCapacity': 'TOTAL', 
          'ProjectionExpression': 'ProductCategory, Price, Title'
         }

response = get_item_db(table_name=product_catalog_table['table_name'], 
                    key={'Id': {'N': '101'}}, **kwargs
                    )
print(response)

#### Fin de la Parte Opcional - 2


<a id='5-3'></a>
### 5.3 - Consultar elementos que comparten la misma clave de partición

En DynamoDB, una colección de elementos es un grupo de elementos que comparten el mismo valor de clave de partición, lo que significa que los elementos están relacionados. Puede consultar los elementos que pertenecen a una colección de elementos (es decir, que tienen la misma clave de partición) utilizando el método [DynamoDB query()](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb/client/query.html). En este método, es necesario especificar el valor particular de la clave de partición de los elementos de interés.

Las colecciones de elementos solo existen en tablas que tienen tanto una Clave de Partición como una Clave de Ordenamiento. Opcionalmente, puede proporcionar al método de consulta un atributo de clave de ordenamiento y utilizar un operador de comparación para refinar los resultados de la búsqueda.

En el siguiente ejercicio, utilizarás la tabla `Reply` ya que tiene tanto una Clave de Partición como una Clave de Ordenamiento. Primero, veamos su contenido.


In [ ]:
response = scan_db(reply_table['table_name'])
print(response)

Cada respuesta en esta tabla tiene un `Id` (Clave de partición) que especifica en qué hilo apareció la respuesta dada. Los datos consisten en un total de dos hilos que pertenecen al foro "Amazon DynamoDB", y cada hilo tiene 2 respuestas. Consultemos las respuestas que pertenecen al Hilo 1.

Utilizarás la función `query_db()` definida a continuación. Esta función llama al método `DynamoDB query()` que espera el valor particular de la clave de partición y devuelve todos los elementos que tienen el valor de clave de partición especificado. Puedes asumir que la entrada `kwargs` del método `query_db()` contiene la información necesaria (valor particular de la clave primaria) para el método `DynamoDB query()`.


In [ ]:
def query_db(table_name: str,**kwargs,):
        client = boto3.client("dynamodb")

        try:
            response = client.query(
                TableName=table_name,
                **kwargs,
            )
            logging.info(f"Response {response}")
        except ClientError as e:
            error = e.response.get("Error", {})
            logging.error(
                f"Failed to query DynamoDB. Error: {error.get('Message')}"
            )
            raise
        else:
            logging.info(f"Query result {response.get('Items', {})}")
            return response

Ahora vamos a entrar en los detalles del diccionario `kwargs` que se pasa a `client.query()`.

La siguiente celda muestra un ejemplo de lo que `kwargs` debería contener, como se espera por el método `DynamoDB query()`:

`KeyConditionExpression`: es la condición que especifica el valor de la clave de partición de los elementos que deben ser recuperados; puedes ver en esta sintaxis el nombre de la clave de partición que es `Id` y su valor particular se denota con otro parámetro `:Id` que está definido en el siguiente argumento `ExpressionAttributeValues`. Para entender más sobre esta sintaxis, siempre puedes consultar la [documentación](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb/client/query.html). El parámetro: `ReturnedConsumedCapacity` determina qué nivel de detalle sobre la capacidad consumida debe devolver la respuesta.


In [ ]:
kwargs = {'ReturnConsumedCapacity': 'TOTAL', 
          'KeyConditionExpression': 'Id = :Id',
          'ExpressionAttributeValues': {':Id': {'S': 'Amazon DynamoDB#DynamoDB Thread 1'}}
          } 

# returns the items that has ID = 'Amazon DynamoDB#DynamoDB Thread 1'
response = query_db(table_name=reply_table['table_name'], **kwargs) 
               
print(response)

También puedes consultar los elementos que comparten la misma clave de partición y que también cumplen una cierta condición en la clave de ordenación. Dado que la clave de ordenación de la tabla de Respuestas es una marca de tiempo, puedes agregar una condición a `KeyConditionExpression` para obtener las respuestas de un hilo en particular que se publicaron después de cierto tiempo. Observa más de cerca cómo se compara la clave de ordenación con el parámetro `:ts` y cómo se define este parámetro en `ExpressionAttributeValues`.


In [ ]:
kwargs = {'ReturnConsumedCapacity': 'TOTAL', 
          'KeyConditionExpression': 'Id = :Id and ReplyDateTime > :ts',
          'ExpressionAttributeValues': {':Id': {'S': 'Amazon DynamoDB#DynamoDB Thread 1'}, 
                               ':ts' : {'S':"2015-09-21"}
                               }
          }

response = query_db(table_name=reply_table['table_name'], **kwargs)

print(response)

Además de `keyConditionExpression`, también puedes usar `FilterExpression` para filtrar los resultados basados en atributos que no son clave. Por ejemplo, para encontrar todas las respuestas al Hilo 1 que fueron publicadas por el Usuario B, puedes hacer:


In [ ]:
kwargs = {'ReturnConsumedCapacity': 'TOTAL', 
          'KeyConditionExpression': 'Id = :Id ',
          'FilterExpression': 'PostedBy = :user',
          'ExpressionAttributeValues': {':Id': {'S': 'Amazon DynamoDB#DynamoDB Thread 1'}, 
                               ':user' : {'S':'User B'}
                               }          
          }

response = query_db(table_name=reply_table['table_name'], **kwargs)

print(response)

Ten en cuenta que en la respuesta verás estas líneas:

```
"Count": 1,
"ScannedCount": 2,
```

Esto te indica que la Expresión de Condición de Clave coincidió con 2 elementos (ScannedCount basado en el valor de la clave de partición) y eso es por lo que se te cobró la lectura, pero la Expresión de Filtro redujo el tamaño del conjunto de resultados a 1 elemento (Count).


<a id='ex06'></a>
### Ejercicio 6

Abre la [documentación](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb/client/query.html) para el método `DynamoDB query()` y busca los parámetros `Limit` y `ScanIndexForward`. En este ejercicio, necesitas escribir la siguiente consulta: devolver solo la primera respuesta al hilo 1.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista</b></font>
</summary>
<p>
<ul>
    Considera los parámetros <code>Limit</code> y <code>ScanIndexForward</code>. Si deseas ordenar los elementos en orden ascendente basado en la clave de ordenación, utiliza el parámetro <code>ScanIndexForward</code>. Si deseas limitar el número de elementos, entonces utiliza el parámetro <code>Limit</code>. Esto sería análogo en SQL a: <code>ORDER BY ReplyDateTime ASC LIMIT 1</code>.
   
</ul>
</p>


In [ ]:
kwargs = {'ReturnConsumedCapacity': 'TOTAL', 
          'KeyConditionExpression': 'Id = :Id ',
          'ExpressionAttributeValues': {":Id" : {"S": "Amazon DynamoDB#DynamoDB Thread 1"}},
          ### START CODE HERE ### (~ 2 lines of code)
          'Limit': 1, # @REPLACE           'None': None,
          'ScanIndexForward':  True # @REPLACE           'None': True,
          ### END CODE HERE ###
          }

response = query_db(table_name=reply_table['table_name'], **kwargs)

print(response)

##### __Salida Esperada__ 

**Notas**: 

- El atributo `'ResponseMetadata'` puede variar en tu salida. 
- Los componentes de fecha y hora pueden ser diferentes.

```
{'Items': [{'ReplyDateTime': {'S': '2015-09-15T19:58:22.947Z'}, 'Message': {'S': 'DynamoDB Thread 1 Reply 1 text'}, 'PostedBy': {'S': 'User A'}, 'Id': {'S': 'Amazon DynamoDB#DynamoDB Thread 1'}}], 'Count': 1, 'ScannedCount': 1, 'LastEvaluatedKey': {'Id': {'S': 'Amazon DynamoDB#DynamoDB Thread 1'}, 'ReplyDateTime': {'S': '2015-09-15T19:58:22.947Z'}}, 'ConsumedCapacity': {'TableName': 'de-c2w1-dynamodb-Reply', 'CapacityUnits': 0.5}, 'ResponseMetadata': {'RequestId': 'EMV7RBG34OOUCP0KS4LARC1B6BVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 14 Feb 2024 06:45:11 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '406', 'connection': 'keep-alive', 'x-amzn-requestid': 'EMV7RBG34OOUCP0KS4LARC1B6BVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '284193681'}, 'RetryAttempts': 0}}
```


<a id='ex07'></a>
### Ejercicio 7

Ajusta la consulta para devolver solo la respuesta más reciente para el Hilo 1.


In [ ]:
kwargs = {'ReturnConsumedCapacity': 'TOTAL', 
          'KeyConditionExpression': 'Id = :Id ',
          'ExpressionAttributeValues': {":Id" : {"S": "Amazon DynamoDB#DynamoDB Thread 1"}},
          ### START CODE HERE ### (~ 2 lines of code)
          'Limit': 1, # @REPLACE           'None': None,
          'ScanIndexForward':  False # @REPLACE           'None': False,
          ### END CODE HERE ###
          }

response = query_db(table_name=reply_table['table_name'], **kwargs)

print(response)

##### __Salida Esperada__ 

**Notas**: 

- El atributo `'ResponseMetadata'` puede variar en tu salida. 
- Los componentes de fecha y hora pueden ser diferentes.

```
{'Items': [{'ReplyDateTime': {'S': '2015-09-22T19:58:22.947Z'}, 'Message': {'S': 'Texto de la respuesta 2 del hilo 1 de DynamoDB'}, 'PostedBy': {'S': 'Usuario B'}, 'Id': {'S': 'Amazon DynamoDB#Hilo 1 de DynamoDB'}}], 'Count': 1, 'ScannedCount': 1, 'LastEvaluatedKey': {'Id': {'S': 'Amazon DynamoDB#Hilo 1 de DynamoDB'}, 'ReplyDateTime': {'S': '2015-09-22T19:58:22.947Z'}}, 'ConsumedCapacity': {'TableName': 'de-c2w1-dynamodb-Reply', 'CapacityUnits': 0.5}, 'ResponseMetadata': {'RequestId': '7V1KQ1STK5C07EFR1DK0PGPBKFVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 14 Feb 2024 06:45:13 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '406', 'connection': 'keep-alive', 'x-amzn-requestid': '7V1KQ1STK5C07EFR1DK0PGPBKFVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '1848739007'}, 'RetryAttempts': 0}}
```


<a id='5-4'></a>
### 5.4 - Filtrando las exploraciones de tabla

El método `DynamoDB scan()` es similar al método `DynamoDB query()` excepto que estás explorando toda la tabla, no solo una única colección de elementos, por lo que no hay una Expresión de Condición de Clave que necesites especificar para `DynamoDB scan()`. Sin embargo, puedes especificar una `FilterExpression` que reducirá el tamaño del conjunto de resultados (aunque no reducirá la cantidad de capacidad consumida).

Por ejemplo, encuentra todas las respuestas en la tabla Reply que fueron publicadas por el Usuario A:


In [ ]:
kwargs = {'ReturnConsumedCapacity': 'TOTAL', 
          'FilterExpression': 'PostedBy = :user', 
          'ExpressionAttributeValues': {':user' : {'S':'User A'}}
        }

response = scan_db(reply_table['table_name'], **kwargs)
print(response)

La respuesta contiene estos campos:

```
"Count": 3,
"ScannedCount": 4,
```

Esto te informa que el `DynamoDB scan()` escaneó los 4 elementos (`ScannedCount`) en la tabla y eso es lo que se te cobró por leer, pero la `FilterExpression` redujo el tamaño del conjunto de resultados a 3 elementos (`Count`).


#### Inicio de la Parte Opcional - 3 (Última clave evaluada)


Al escanear datos, la respuesta puede exceder el límite de 1MB en el lado del servidor o superar el parámetro `Limit` especificado. En tales casos, la respuesta del escaneo contendrá un campo `LastEvaluatedKey`, lo que permitirá que las llamadas de escaneo subsiguientes continúen desde donde el escaneo anterior se detuvo. Por ejemplo, si el escaneo inicial identificó 3 elementos en el conjunto de resultados, ejecutarlo nuevamente con un límite máximo de elementos de 2 puede demostrar este comportamiento.


In [ ]:
kwargs = {'ReturnConsumedCapacity': 'TOTAL', 
          'FilterExpression': 'PostedBy = :user', 
          'ExpressionAttributeValues': {':user' : {'S':'User A'}},
          'Limit': 2
        }
response = scan_db(reply_table['table_name'], **kwargs)
print(response)

Tomemos el campo `LastEvaluatedKey` y úselo para la siguiente exploración de la tabla:


In [ ]:
last_evaluated_key = response.get("LastEvaluatedKey")
print(last_evaluated_key)

Por lo tanto, puedes invocar la solicitud de escaneo nuevamente, esta vez pasando ese valor `LastEvaluatedKey` al parámetro `ExclusiveStartKey`:


In [ ]:
kwargs = {'ReturnConsumedCapacity': 'TOTAL', 
          'FilterExpression': 'PostedBy = :user', 
          'ExpressionAttributeValues': {':user' : {'S':'User A'}},
          'Limit': 2,
          'ExclusiveStartKey': last_evaluated_key
        }

response = scan_db(reply_table['table_name'], **kwargs)
print(response)

Verifique los datos en la tabla del Foro con un comando de escaneo para devolver solo los Foros que tienen más de 1 hilo y más de 50 vistas.

Puede ver que algunos elementos tienen un atributo de número de Hilos y un atributo de número de Vistas. Para resolver este problema, desea utilizar esos atributos en la `FilterExpression`. Asegúrese de especificar que estos valores son del tipo Número usando "N" en el parámetro `--expression-attribute-values`.

Dado que el nombre del atributo `Vistas` es una Palabra Reservada de DynamoDB, DynamoDB le brinda la capacidad de poner un marcador de posición en la `FilterExpression` y proporcionar el nombre de atributo real en el parámetro CLI `--expression-attribute-names`. Para obtener más información, consulte [Nombres de Atributos de Expresión](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/Expressions.ExpressionAttributeNames.html) en DynamoDB en la Guía para Desarrolladores.


In [ ]:
kwargs = {'ReturnConsumedCapacity': 'TOTAL', 
          'FilterExpression': 'Threads >= :threads AND #Views >= :views', 
          'ExpressionAttributeValues': {":threads" : {"N": "1"},
                                        ":views" : {"N": "50"}},
          'ExpressionAttributeNames':{"#Views" : "Views"}
        }

response = scan_db(forum_table['table_name'], **kwargs)
print(response)    

#### Fin de la Parte Opcional - 3


<a id='6'></a>
## 6 - Insertar y Actualizar Datos


<a id='6-1'></a>
### 6.1 - Insertar datos

El método `put_item()` de DynamoDB se utiliza para crear un nuevo ítem o reemplazar ítems existentes con un nuevo ítem. Ya has creado la función `put_item_db()` para cargar datos ítem por ítem en algunas tablas. Ahora, digamos que queremos insertar un nuevo ítem en la tabla Reply. Verás en la respuesta que esta solicitud consumió 1 Unidad de Capacidad de Escritura (WCU) (Una unidad de capacidad de escritura representa una escritura por segundo para un ítem de hasta 1 KB de tamaño. [referencia](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/HowItWorks.ReadWriteCapacityMode.html)).


In [ ]:
new_item = {
        "Id" : {"S": "Amazon DynamoDB#DynamoDB Thread 2"},
        "ReplyDateTime" : {"S": "2021-04-27T17:47:30Z"},
        "Message" : {"S": "DynamoDB Thread 2 Reply 3 text"},
        "PostedBy" : {"S": "User C"}
    }

kwargs = {'ReturnConsumedCapacity': 'TOTAL'}
    

response = put_item_db(table_name=reply_table["table_name"], item=new_item, **kwargs)
print(response)

<a id='6-2'></a>
### 6.2 - Actualizar datos

El método `update_item()` de DynamoDB se puede utilizar para editar los atributos de un elemento existente o agregar un nuevo elemento a la tabla si aún no existe. "Este método requiere que proporciones la clave principal del elemento que deseas actualizar. También debes proporcionar una expresión de actualización (`UpdateExpression`), indicando los atributos que deseas modificar y los valores que deseas asignarles" ([guía para desarrolladores](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/WorkingWithItems.html)). Para obtener más información sobre el formato de la expresión de actualización, consulta [aquí](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/Expressions.UpdateExpressions.html). También puedes especificar una expresión de condición para determinar qué elementos deben ser modificados.
Echa un vistazo a la función proporcionada a continuación, donde `ReturnValues='UPDATED_NEW'` devuelve solo los atributos actualizados tal como aparecen después de la operación de actualización.


In [ ]:
def update_item_db(table_name: str, key: Dict[str, Any], **kwargs):
    client = boto3.client("dynamodb")

    response = client.update_item(
        TableName=table_name, Key=key, ReturnValues="UPDATED_NEW", **kwargs
    )

    return response

In [ ]:
kwargs= {    
    'UpdateExpression': 'SET Messages = :newMessages',
    'ConditionExpression': 'Messages = :oldMessages',
    'ExpressionAttributeValues': {
        ":oldMessages" : {"N": "4"},
        ":newMessages" : {"N": "5"}
    }
}
response = update_item_db(table_name=forum_table['table_name'], key={'Name' : {'S': 'Amazon DynamoDB'}}, **kwargs)
print(response)

Esta función actualizó los Foros que tenían un total de 4 mensajes para que ahora estos foros tengan 5 mensajes.


<a id='ex08'></a>
### Ejercicio 8

Actualiza el elemento `de-c2w1-dynamodb-ProductCatalog` con `Id="201"` para agregar los nuevos colores "Azul" y "Amarillo" a la lista de colores para ese tipo de bicicleta. Se te proporciona la expresión de actualización que consiste en agregar a una lista de valores. Para obtener más información, puedes consultar la página de <a href="https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/Expressions.UpdateExpressions.html">Expresiones de Actualización</a> en la Guía del Desarrollador que tiene secciones sobre Agregar y Eliminar Elementos en una Lista.

Puedes utilizar `DynamoDB get_item()` para verificar que estos cambios se hayan realizado después de cada paso.


In [ ]:
kwargs = {
    'UpdateExpression': 'SET #Color = list_append(#Color, :values)',
    'ExpressionAttributeNames': {'#Color': 'Color'},
    'ExpressionAttributeValues': {':values': {'L': [{'S': 'Blue'}, {'S': 'Yellow'}]}},
    'ReturnConsumedCapacity': 'TOTAL'
}

### START CODE HERE ### (~ 1 line of code)
response = update_item_db(table_name=product_catalog_table['table_name'], key={'Id': {'N': '201'}}, **kwargs) # @REPLACE EQUALS update_item_db(table_name=None['None'], key={'Id': {'N': 'None'}}, None)
### END CODE HERE ###

print(response)

##### __Salida Esperada__

**Nota**: Los componentes de fecha y hora pueden ser diferentes.

```
{'Attributes': {'Color': {'L': [{'S': 'Rojo'}, {'S': 'Negro'}, {'S': 'Azul'}, {'S': 'Amarillo'}]}}, 'ConsumedCapacity': {'TableName': 'de-c2w1-dynamodb-ProductCatalog', 'CapacityUnits': 1.0}, 'ResponseMetadata': {'RequestId': 'D3CK6H42KF2DB9ITOO8K7UD95FVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mié, 14 Feb 2024 06:45:33 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '173', 'connection': 'keep-alive', 'x-amzn-requestid': 'D3CK6H42KF2DB9ITOO8K7UD95FVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '3394722026'}, 'RetryAttempts': 0}}
```


In [ ]:
response = get_item_db(table_name=product_catalog_table['table_name'], 
                                      key={'Id': {'N': '201'}}
                                    )
print(response)

<a id='ex09'></a>
### Ejercicio 9

En este ejercicio, utiliza la función `update_item_db()` para eliminar las entradas de la lista "Blue" y "Yellow" que acabas de agregar, para devolver el elemento de la bicicleta a su estado original. En DynamoDB, las listas tienen índices basados en 0.


In [ ]:
kwargs = {
    'UpdateExpression': 'REMOVE #Color[2], #Color[3]',
    'ExpressionAttributeNames': {'#Color': 'Color'},
    'ReturnConsumedCapacity': 'TOTAL'
}

### START CODE HERE ### (~ 1 line of code)
response = update_item_db(table_name=product_catalog_table['table_name'], key={'Id': {'N': '201'}}, **kwargs) # @REPLACE EQUALS update_item_db(table_name=None['None'], key={'Id': {'N': 'None'}}, None)
### END CODE HERE ###

print(response)

##### __Salida Esperada__

**Nota**: Los componentes de fecha y hora pueden ser diferentes.

```
{'ConsumedCapacity': {'TableName': 'de-c2w1-dynamodb-ProductCatalog', 'CapacityUnits': 1.0}, 'ResponseMetadata': {'RequestId': 'CFU1SD9DECJOS6SLMVT3KT4CH7VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 14 Feb 2024 06:45:37 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '88', 'connection': 'keep-alive', 'x-amzn-requestid': 'CFU1SD9DECJOS6SLMVT3KT4CH7VV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '866229524'}, 'RetryAttempts': 0}}
```


In [ ]:
response = get_item_db(table_name=product_catalog_table['table_name'], 
                                      key={'Id': {'N': '201'}}
                                    )
print(response)

<a id='7'></a>
## 7 - Eliminar datos

El método `DynamoDB DeleteItem()` se utiliza para eliminar un elemento. Las eliminaciones en DynamoDB son operaciones singleton. No hay un comando único que pueda ejecutar que eliminaría todas las filas en la tabla. Eliminemos uno de los elementos que agregamos previamente a la tabla Reply; para eso, necesitas hacer referencia a la Clave Primaria completa. Recuerda que la tabla Reply tiene `Id` como clave de partición y `ReplyDateTime` como clave de ordenación, por lo que la Clave Primaria completa está compuesta por esas dos claves. Sigue las instrucciones para crear la función `delete_item_db()`.


<a id='ex10'></a>
### Ejercicio 10

1. Crea el objeto Cliente `client`.
2. Utiliza el método `client.delete_item()` del objeto cliente. Asegúrate de agregar el nombre de la tabla y los parámetros clave en la llamada al método. El resto de los parámetros deben pasarse como argumentos de palabra clave.


In [ ]:
def delete_item_db(table_name: str, key: dict[str, Any], **kwargs):
    ### START CODE HERE ### (~ 2 lines of code)
    client = boto3.client("dynamodb") # @REPLACE EQUALS None
    response = client.delete_item(TableName=table_name, Key=key, **kwargs) # @REPLACE EQUALS client.delete_item(TableName=None, Key=None, None)
    ### END CODE HERE ###
    
    logging.info(f"response {response}")

In [ ]:
key = {"Id" : {"S": "Amazon DynamoDB#DynamoDB Thread 2"},
       "ReplyDateTime" : {"S": "2021-04-27T17:47:30Z"}
       }

delete_item_db(table_name=reply_table['table_name'], key=key)

El mismo elemento puede ser eliminado más de una vez. Puedes ejecutar el mismo comando anterior tantas veces como desees y no reportará ningún error: incluso si la clave no existe, el método `DynamoDB delete_item()` devuelve éxito. Ahora, debes decrementar el recuento relacionado de *Mensajes* del Foro.


In [ ]:
kwargs= {    
    'UpdateExpression': 'SET Messages = :newMessages',
    'ConditionExpression': 'Messages = :oldMessages',
    'ExpressionAttributeValues': {
        ":oldMessages" : {"N": "5"},
        ":newMessages" : {"N": "4"}
    },
    'ReturnConsumedCapacity': 'TOTAL'
}

update_item_db(table_name=forum_table['table_name'], key={'Name' : {'S': 'Amazon DynamoDB'}}, **kwargs)


La siguiente sección es completamente opcional. Siéntete libre de saltarla para pasar a la última sección que trata sobre la limpieza.


<a id='8'></a>
## 8 - Transacciones - Sección opcional


El `DynamoDB transact_write_items` es una operación de escritura síncrona que agrupa hasta 100 solicitudes de acción, con un límite de tamaño colectivo de 4MB para toda la transacción. Estas acciones pueden operar en elementos de varias tablas, aunque no a través de cuentas o regiones de AWS distintas. Además, ninguna de las dos acciones puede apuntar al mismo elemento. La ejecución de las acciones es atómica, asegurando que todas tengan éxito o todas fallen.

Has visto que los datos de muestra incluyen tablas interconectadas: `Forum`, `Thread` y `Reply`. Al agregar un nuevo elemento `Reply`, es necesario incrementar el contador de `Messages` en el elemento `Forum` asociado. Esta operación debe ocurrir dentro de una transacción para garantizar que ambos cambios tengan éxito o fallen simultáneamente. Cualquier observador que lea estos datos debería presenciar ambos cambios o ninguno al mismo tiempo.

Las transacciones de DynamoDB se adhieren al concepto de **idempotencia**, permitiendo el envío de la misma transacción varias veces. Sin embargo, DynamoDB la ejecutará solo una vez. Esta característica es particularmente valiosa al trabajar con APIs que carecen de idempotencia inherente, como al usar `update_item` para modificar un campo numérico. Durante la ejecución de la transacción, especificas una cadena como el `ClientRequestToken` (también conocido como Token de Idempotencia).


<a id='ex11'></a>
### Ejercicio 11

1. Crear el objeto Cliente `client`.
2. Llamar al método `DynamoDB transact_write_items()`; pasar explícitamente los elementos de transacción. Otros parámetros deben pasarse como parámetros de palabras clave.


In [ ]:
def transact_write_items_db(transaction_items: List[Dict[str, Any]], **kwargs):
    ### START CODE HERE ### (~ 2 lines of code)
    client = boto3.client("dynamodb") # @REPLACE EQUALS None
    response = client.transact_write_items(TransactItems=transaction_items, **kwargs) # @REPLACE EQUALS client.transact_write_items(TransactItems=None, None)
    ### END CODE HERE ###

    return response

Realicemos primero la transacción agregando un nuevo usuario a la tabla `Reply`.


In [ ]:
transaction_items=[
    {
        "Put": {
            "TableName" : reply_table['table_name'],
            "Item" : {
                "Id" : {"S": "Amazon DynamoDB#DynamoDB Thread 2"},
                "ReplyDateTime" : {"S": "2021-04-27T17:47:30Z"},
                "Message" : {"S": "DynamoDB Thread 2 Reply 3 text"},
                "PostedBy" : {"S": "User C"}
            }
        }
    },
    {
        "Update": {
            "TableName" : forum_table['table_name'],
            "Key" : {"Name" : {"S": "Amazon DynamoDB"}},
            "UpdateExpression": "ADD Messages :inc",
            "ExpressionAttributeValues" : { ":inc": {"N" : "1"} }
        }
    }
]

kwargs = {'ClientRequestToken': 'TRANSACTION1'}

response = transact_write_items_db(transaction_items=transaction_items, **kwargs)
print(response)

Después de que la transacción haya finalizado, puedes echar un vistazo al artículo del Foro y verás que el recuento de Mensajes se incrementó en 1, de 4 a 5.


In [ ]:
response = get_item_db(table_name=forum_table['table_name'], key={"Name" : {"S": "Amazon DynamoDB"}})
print(response)

Si la transacción se ejecuta nuevamente con el mismo valor del `'ClientRequestToken'` como `'TRANSACTION1'`, puedes ver que se ignoran otras invocaciones de la transacción y el atributo `Messages` permanece con el valor en 5. También puedes usar transacciones para revertir la operación realizada anteriormente; ten en cuenta que hay un nuevo valor para el `ClientRequestToken` de esta transacción:


In [ ]:
transaction_items=[
    {
        "Delete": {
            "TableName" : reply_table['table_name'],
            "Key" : {
                "Id" : {"S": "Amazon DynamoDB#DynamoDB Thread 2"},
                "ReplyDateTime" : {"S": "2021-04-27T17:47:30Z"}
            }
        }
    },
    {
        "Update": {
            "TableName" : forum_table['table_name'],
            "Key" : {"Name" : {"S": "Amazon DynamoDB"}},
            "UpdateExpression": "ADD Messages :inc",
            "ExpressionAttributeValues" : { ":inc": {"N" : "-1"} }
        }
    }
]

kwargs = {'ClientRequestToken': 'TRANSACTION2'}

response = transact_write_items_db(transaction_items=transaction_items, **kwargs)
print(response)

<a id='9'></a>
## 9 - Limpieza

Eliminar las tablas creadas de DynamoDB. Verifique la función proporcionada en la siguiente celda `delete_table_db()` y ejecute las siguientes celdas para eliminar las tablas.


In [ ]:
def delete_table_db(table_name: str):
        client = boto3.client("dynamodb")
        response = client.delete_table(TableName=table_name)
        return response

In [ ]:
for dynamodb_tab in [product_catalog_table, forum_table, reply_table, thread_table]:
    response = delete_table_db(table_name=dynamodb_tab['table_name'])
    print(response)

Finalmente, puedes ir a la Consola de AWS, buscar **DynamoDB**, hacer clic en Tablas y verificar que las tablas hayan sido eliminadas.
